# Using a plain logistic regression in SciPy

### Importing Libraries

In [30]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from math import log, exp
from scipy.stats import chi2
from sklearn.model_selection import KFold, cross_val_score

### Loading data

In [20]:
df = pd.read_csv('https://bit.ly/33ebs2R', delimiter=",")

### Logistic Regression

Extract input variables (all rows, all columns but last column)

In [21]:
X = df.values[:, :-1]

Extract output column (all rows, last column)

In [22]:
Y = df.values[:, -1]

Perform logistic regression

Turn off penalty

In [23]:
model = LogisticRegression(penalty=None)
model.fit(X, Y)

LogisticRegression(penalty=None)

print beta1

In [24]:
b1 = model.coef_.flatten()
b1

array([0.69267212])

print beta0

In [25]:
b0 = model.intercept_.flatten()
b0

array([-3.17576395])

## Calculating the R² for a logistic regression

In [27]:
patient_data = list(pd.read_csv('https://bit.ly/33ebs2R', delimiter=",") \
                                .itertuples())

# Declare fitted logistic regression
b0 = -3.17576395
b1 = 0.69267212

def logistic_function(x):
    p = 1.0 / (1.0 + exp(-(b0 + b1 * x)))
    return p

# calculate the log likelihood of the fit
log_likelihood_fit = sum(log(logistic_function(p.x)) * p.y +
                         log(1.0 - logistic_function(p.x)) * (1.0 - p.y)
                         for p in patient_data)

# calculate the log likelihood without fit
likelihood = sum(p.y for p in patient_data) / len(patient_data)

log_likelihood = sum(log(likelihood) * p.y + log(1.0 - likelihood) * (1.0 - p.y) \
	for p in patient_data)

# calculate R-Square
r2 = (log_likelihood - log_likelihood_fit) / log_likelihood

print(r2) 

0.306456105756576


OK, so we got an R² = 0.306456, so do hours of chemical exposure explain whether someone shows symptoms? A poor fit will be closer to an R² of 0.0 and a greater fit will be closer to 1.0. Therefore, we can conclude that hours of exposure is mediocre for predicting symptoms, as the R² is 0.30645. There must be variables other than time exposure that better predict if someone will show symptoms. This makes sense because we have a large mix of patients showing symptoms versus not showing symptoms for most of our observed data.

## P-values

Just like linear regression, we are not done just because we have an R². We need to investigate how likely we would have seen this data by chance rather than because of an actual relationship. This means we need a p-value.

To do this, we will need to learn a new probability distribution called the chi-square distribution, annotated as X² distribution. It is continuous and used in several areas of statistics, including this one!

In [29]:
# calculate p-value
chi2_input = 2 * (log_likelihood_fit - log_likelihood)
p_value = chi2.pdf(chi2_input, 1) # 1 degree of freedom (n - 1)

print(p_value)  # 0.0016604875618753787

0.0016604875618753787


So we have a p-value of 0.00166, and if our threshold for signifiance is .05, we say this data is statistically significant and was not by random chance.